# Deep Convolutional Generative Adversarial Network

The code is written using the [Keras Sequential API](https://www.tensorflow.org/guide/keras) with a `tf.GradientTape` training loop.

### Import TensorFlow and other libraries

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

In [ ]:
tf.__version__

In [ ]:
# To generate GIFs
!pip install imageio

In [ ]:
!pip install pillow matplotlib

In [ ]:
import imageio
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from glob import glob
from tensorflow.keras import layers, models
import time
from IPython import display
import random
import my_helper

In [ ]:
# image_count = os.listdir('img_align_celeba/')
# print(len(image_count))

# img_detectFace = []
# image_size = 64

# cascPath = "haarcascade_frontalface_default.xml"
# # Create the haar cascade
# faceCascade = cv2.CascadeClassifier(cascPath)

# for i in range(len(image_count)):
#     if i % 10000 == 9999:
#         print(str(i+1) + " processed images")
#     filename = 'img_align_celeba/{:06d}.jpg'.format(i+1)
#     img = plt.imread(filename)
    
#     # Detect frontalface
#     faces = faceCascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=40)
#     if len(faces) == 1:
#         img_detectFace.append(filename)

# print(len(img_detectFace))
# images = my_helper.get_batch(img_detectFace, image_size, image_size, 'RGB')
# print(images.shape)

In [ ]:
# print(images.shape)
# for i in range(images.shape[0]):
#     if i % 10000 == 9999:
#         print("Save {:06d}th image".format(i+1))
#     img = images[i].astype(np.uint8)
#     plt.imsave('frontalface/{:06d}.jpg'.format(i+1), img)

In [ ]:
image_list = os.listdir('frontalface/')
print(len(image_list))

In [ ]:
images = []
for i in range(len(image_list)):
    img = plt.imread('frontalface/{:06d}.jpg'.format(i+1))
    images.append(img)

In [ ]:
print(len(images))
print(type(images))

In [ ]:
images = np.array(images).astype(np.float32)
print(images.shape)
images = images / 127.5 - 1.0 # Normalize the images to [-1, 1]

### Load and prepare the dataset


In [ ]:
BATCH_SIZE = 256
BUFFER_SIZE = len(images)
print(BUFFER_SIZE)

In [ ]:
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## Create the models

Both the generator and discriminator are defined using the [Keras Sequential API](https://www.tensorflow.org/guide/keras#sequential_model).

### The Generator

The generator uses `tf.keras.layers.Conv2DTranspose` (upsampling) layers to produce an image from a seed (random noise). Start with a `Dense` layer that takes this seed as input, then upsample several times until you reach the desired image size of 64x64x3. Notice the `tf.keras.layers.LeakyReLU` activation for each layer, except the output layer which uses tanh.

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(4*4*1024, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization(epsilon=0.00005))
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Reshape((4, 4, 1024)))
    assert model.output_shape == (None, 4, 4, 1024) # Note: None is the batch size

    model.add(layers.Dropout(0.3))
    # 4x4x1024 -> 8x8x512
    model.add(layers.Conv2DTranspose(512, (5, 5), strides=(2, 2), padding='same', use_bias=False,
                                     kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)))
    assert model.output_shape == (None, 8, 8, 512)
    model.add(layers.BatchNormalization(epsilon=0.00005))
    model.add(layers.LeakyReLU(alpha=0.2))
    
    model.add(layers.Dropout(0.3))
    # 8x8x512 -> 16x16x256
    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False,
                                     kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)))
    assert model.output_shape == (None, 16, 16, 256)
    model.add(layers.BatchNormalization(epsilon=0.00005))
    model.add(layers.LeakyReLU(alpha=0.2))
    
    model.add(layers.Dropout(0.3))
    # 16x16x256 -> 32x32x128
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False,
                                     kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)))
    assert model.output_shape == (None, 32, 32, 128)
    model.add(layers.BatchNormalization(epsilon=0.00005))
    model.add(layers.LeakyReLU(alpha=0.2))

    # 32x32x128 -> 64x64x3
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 64, 64, 3)

    return model

Use the (as yet untrained) generator to create an image.

In [ ]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)
print(type(generated_image))
print(generated_image.shape)
plt.imshow(((generated_image[0].numpy() + 1.0) * 127.5).astype(np.uint8))

### The Discriminator

The discriminator is a CNN-based image classifier.

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    # 64x64x3 -> 32x32x64
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                            kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02), input_shape=[64, 64, 3]))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    # 32x32x64 -> 16x16x128
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same',
                            kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    # 16x16x128 -> 8x8x256
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same',
                            kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

Use the (as yet untrained) discriminator to classify the generated images as real or fake. The model will be trained to output positive values for real images, and negative values for fake images.

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print(decision)

## Define the loss and optimizers

Define loss functions and optimizers for both models.


In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Discriminator loss

This method quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predictions on real images to an array of 1s, and the discriminator's predictions on fake (generated) images to an array of 0s.

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output)*random.uniform(0.9, 1.0), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss
The generator's loss quantifies how well it was able to trick the discriminator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1). Here, we will compare the discriminators decisions on the generated images to an array of 1s.

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output)*random.uniform(0.9, 1.0), fake_output)

The discriminator and the generator optimizers are different since we will train two networks separately.

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)

### Save checkpoints
This notebook also demonstrates how to save and restore models, which can be helpful in case a long running training task is interrupted.

In [ ]:
from datetime import datetime
current_time = datetime.now()
# year month day hour minute second microsecond
print()

In [ ]:
current_time = datetime.now()
current_time_naming = "{:02d}".format(current_time.month) + \
                      "{:02d}".format(current_time.day) +  "_" + \
                      "{:02d}".format(current_time.hour) + \
                      "{:02d}".format(current_time.minute) + \
                      "{:02d}".format(current_time.second)
os.mkdir(current_time_naming)
print(current_time_naming)
checkpoint_dir = "./training_checkpoints-" + current_time_naming
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop



In [ ]:
EPOCHS = 100
noise_dim = 100
num_examples_to_generate = 16

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

The training loop begins with generator receiving a random seed as input. That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
        
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    # generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    
    tf.cond(gen_loss > disc_loss, 
            lambda: generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables)), 
            lambda: discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables)))

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        count = 0
        for image_batch in dataset:
            count += 1
            if count % 100 == 0:
                print("Batch {:02d}".format(count))
            train_step(image_batch)
            
        # Produce images for the GIF as we go
        #display.clear_output(wait=True)
        generate_and_save_images(generator,
                                 epoch + 1,
                                 seed)
        
        # Save the model every 10 epochs
        if (epoch + 1) % 10 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
            
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
        
    # Generate after the final epoch
    #display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epochs,
                             seed)

**Generate and save images**



In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4,4))
    
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(((predictions[i].numpy() + 1.0) * 127.5).astype(np.uint8))
        plt.axis('off')
        
    plt.savefig(current_time_naming + '/image_{:03d}.png'.format(epoch))
    plt.show()

## Train the model
Call the `train()` method defined above to train the generator and discriminator simultaneously. Note, training GANs can be tricky. It's important that the generator and discriminator do not overpower each other (e.g., that they train at a similar rate).

At the beginning of the training, the generated images look like random noise. As training progresses, the generated images will look increasingly real. This may take about one minute / epoch with the default settings on Colab.

In [ ]:
%%time
train(train_dataset, EPOCHS)

Restore the latest checkpoint.

In [ ]:
loading_checkpoint = "./training_checkpoints-" + current_time_naming
checkpoint.restore(tf.train.latest_checkpoint(loading_checkpoint))

In [ ]:
result_folder = './final_result-' + current_time_naming
os.mkdir(result_folder)

In [ ]:
for i in range(500):
    noise = tf.random.normal([9, 100])
    test_image = generator(noise, training=False)
    images_array = test_image.numpy()
    my_helper.output_fig(images_array, result_folder + '/' + '{:03d}_image'.format(i+1))

In [ ]:
noise = tf.random.normal([9, 100])
test_image = generator(noise, training=False)
images_array = test_image.numpy()
print(images_array.shape)
my_helper.output_fig(images_array, 'test_image')

## Create a GIF


In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
    return PIL.Image.open(current_time_naming + '/image_{:03d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

Use `imageio` to create an animated gif using the images saved during training.

In [ ]:
anim_file = 'celebA_face.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob(os.path.join(current_time_naming, 'image*.png'))
    filenames = sorted(filenames)
    last = -1
    for i,filename in enumerate(filenames):
        frame = 2*(i**0.5)
        if round(frame) > round(last):
            last = frame
        else:
            continue
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

import IPython
if IPython.version_info > (6,2,0,''):
    display.Image(filename=anim_file)

Compress the 500 images with the code below:

In [ ]:
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

if __name__ == '__main__':
    zipf = zipfile.ZipFile('result_image-1105_131845.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir(result_folder, zipf)
    zipf.close()